In [13]:
from IPython.display import HTML, display
from copy import deepcopy
import random
import os
import openai
import time
import sys
import ast
import json

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from getpass import getpass
import os
import pandas as pd

import pdfkit
import html
from copy import deepcopy
import re

In [2]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

········


### Setup Model

In [3]:
model = ChatOpenAI(model="gpt-4-0613")
model.temperature = 0.8

### DBBench Environment

In [4]:
environment_prompt_template = '''
Pretend you are a MySQL database, responding to SQL statements from an agent. Provide realistic MySQL outputs for SELECT, INSERT, UPDATE, and DELETE operations, maintaining the state of the simulated database accordingly.  The user is expecting answers like those that would be received when using  mysql-connector-python. Reflect changes in subsequent outputs, and confirm operations with typical MySQL success messages. The initial state of the database is described below

Initial Database state:
{}

First command:
{}

Please only respond in rawMySQL format (with no extra formatting or commentary) for a user of  mysql-connector-python, for example, if the result is 59.555, the result would be presented as [('59.555',)]. After responding, end your response.
'''

mysql_agent_prompt = '''I will ask you a question, then you should help me operate a MySQL database with SQL to answer the question. You have to explain the problem and your solution to me and write down your thoughts. After thinking and explaining thoroughly, every round you can choose to operate or to answer. your operation should be like this: Action: Operation ```sql SELECT * FROM table WHERE condition; ``` You MUST put SQL in markdown format without any other comments. Your SQL should be in one line. Every time you can only execute one SQL statement. I will only execute the statement in the first SQL code block. Every time you write a SQL, I will execute it for you and give you the output. If you are done operating, and you want to commit your final answer, then write down: Action: Answer Final Answer: ["ANSWER1", "ANSWER2", ...] DO NOT write this pattern unless you are sure about your answer. I expect an accurate and correct answer. Your answer should be accurate. Your answer must be exactly the same as the correct answer. If the question is about modifying the database, then after done operation, your answer field can be anything. If your response cannot match any pattern I mentioned earlier, you will be judged as FAIL immediately. Your input will be raw MySQL response, you have to deal with it by yourself.'''

In [8]:
task = '''How many French drivers who obtain the laptime less than 01:00.00? There are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.'''
agent_messages = [
    HumanMessage(content=mysql_agent_prompt),
    AIMessage(content="Ok."),
    HumanMessage(content=task)
]


In [9]:
agent_response = model.predict_messages(agent_messages)

In [10]:
agent_response.content

"In order to answer this question, we first need to join the two tables on the driverId field. This will give us a new table where each row contains the information about one lap for one driver, including the driver's nationality and the laptime.\n\nSecond, we need to filter this table to only include rows where the nationality is 'French' and the laptime is less than '01:00.00'.\n\nThe problem is that the laptime is stored in milliseconds in the lapTimes table. So we need to convert '01:00.00' to milliseconds. '01:00.00' is one minute, which is equal to 60,000 milliseconds.\n\nTherefore, our SQL statement should look something like this:\n\n```sql\nSELECT COUNT(*) \nFROM drivers d \nJOIN lapTimes lt ON d.driverId = lt.driverId \nWHERE d.nationality = 'French' AND lt.milliseconds < 60000;\n```\n\nWith this SQL query, we're joining the drivers and lapTimes tables on the driverId field, filtering for French drivers and laps shorter than one minute, and finally counting the number of such

In [13]:
import re
first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
if first_sql_block:
    sql_code = first_sql_block.group(1).strip()
else:
    sql_code = "No SQL code block found"
sql_code

"SELECT COUNT(*) \nFROM drivers d \nJOIN lapTimes lt ON d.driverId = lt.driverId \nWHERE d.nationality = 'French' AND lt.milliseconds < 60000;"

In [18]:
environment_information = task.split("?")[1].strip()

In [19]:
environment_prompt = environment_prompt_template.format(environment_information, sql_code)

environment_messages = [
    HumanMessage(content=environment_prompt)
]

In [20]:
environment_result = model.predict_messages(environment_messages)

In [21]:
environment_result.content

"[('15',)]"

In [22]:
agent_messages.append(HumanMessage(content=environment_result.content))

In [24]:
num_turns = 10

for i in range(num_turns):
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_result = agent_response.content.split("Action:")[1].strip()
    environment_messages.append(HumanMessage(content=agent_result))
    agent_messages.append(agent_response)
    if "Final Answer:" in agent_result:
        break
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    print(environment_result.content)
    agent_messages.append(HumanMessage(content=environment_result.content))

The output suggests there are 15 records that fulfill some condition. But we need to know what that condition was. This seems like either the result of a previous query or a confusion. 

Here's what we want to do:

Since we want to know how many French drivers have achieved a lap time less than 01:00.00 (which I will assume is in the format minutes:seconds and correlates with the 'milliseconds' field in the lapTimes table), we need to:

1. Join the 'drivers' and 'lapTimes' tables on the 'driverId'. This will give us a combined table where we have both driver information and their respective lap times.
2. Filter this combined table to only include drivers of French nationality.
3. Further filter this table to only include lap times that are less than 1 minute, which is 60000 milliseconds.
4. Count the unique driverId that met these conditions.

Let's execute the operation to get these records:

Action: Operation
```sql
SELECT COUNT(DISTINCT d.driverId) 
FROM drivers d 
JOIN lapTimes l O

#### Get tasks from AgentInstruct Data

In [6]:
import pandas as pd
df = pd.read_parquet('~/spikes/notebook_spikes/agentinstruct/db-00000-of-00001-916a87c4725da8c0.parquet')
print(df)

                                         conversations      id
0    [{'from': 'human', 'loss': None, 'value': 'I w...    db_0
1    [{'from': 'human', 'loss': None, 'value': 'I w...    db_1
2    [{'from': 'human', 'loss': None, 'value': 'I w...    db_2
3    [{'from': 'human', 'loss': None, 'value': 'I w...    db_3
4    [{'from': 'human', 'loss': None, 'value': 'I w...    db_4
..                                                 ...     ...
533  [{'from': 'human', 'loss': None, 'value': 'I w...  db_533
534  [{'from': 'human', 'loss': None, 'value': 'I w...  db_534
535  [{'from': 'human', 'loss': None, 'value': 'I w...  db_535
536  [{'from': 'human', 'loss': None, 'value': 'I w...  db_536
537  [{'from': 'human', 'loss': None, 'value': 'I w...  db_537

[538 rows x 2 columns]


In [7]:
print(df.iloc[2]["conversations"][2]['value'])

Among the cards in the set "Hauptset Zehnte Edition", how many of them are designed by Adam Rex?
There are 2 tables involved with this task. The name of the 1st table is cards, and the headers of this table are id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,faceName,flavorName,flavorText,frameEffects,frameVersion,hand,hasAlternativeDeckLimit,hasContentWarning,hasFoil,hasNonFoil,isAlternative,isFullArt,isOnlineOnly,isOversized,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,keywords,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgjsonV4Id,mtgoFoilId,mtgoId,multiverseId,name,number,originalReleaseDate,originalText,originalType,otherFaceIds,power,printings,promoTypes,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,ty

In [8]:
def process(data):
    return (data['conversations'][2]['value'], '\n'.join(data['conversations'][2]['value'].split("\n")[1:]))

In [9]:
process(df.iloc[104])

('How many turnovers per game did the assist champion had in the 2003 NBA season?\nThere are 2 tables involved with this task. The name of the 1st table is players, and the headers of this table are playerID,useFirst,firstName,middleName,lastName,nameGiven,fullGivenName,nameSuffix,nameNick,pos,firstseason,lastseason,height,weight,college,collegeOther,birthDate,birthCity,birthState,birthCountry,highSchool,hsCity,hsState,hsCountry,deathDate,race. The name of the 2nd table is players_teams, and the headers of this table are id,playerID,year,stint,tmID,lgID,GP,GS,minutes,points,oRebounds,dRebounds,rebounds,assists,steals,blocks,turnovers,PF,fgAttempted,fgMade,ftAttempted,ftMade,threeAttempted,threeMade,PostGP,PostGS,PostMinutes,PostPoints,PostoRebounds,PostdRebounds,PostRebounds,PostAssists,PostSteals,PostBlocks,PostTurnovers,PostPF,PostfgAttempted,PostfgMade,PostftAttempted,PostftMade,PostthreeAttempted,PostthreeMade,note.',
 'There are 2 tables involved with this task. The name of the 1s

In [10]:
task_and_environment = df.apply(process, axis=1).tolist()

In [11]:
task_and_environment [200]

('Please list down ID of movies acted by top 5 actors based on actor rating.\nThere are 2 tables involved with this task. The name of the 1st table is actors, and the headers of this table are actorid,a_gender,a_quality. The name of the 2nd table is movies2actors, and the headers of this table are movieid,actorid,cast_num.',
 'There are 2 tables involved with this task. The name of the 1st table is actors, and the headers of this table are actorid,a_gender,a_quality. The name of the 2nd table is movies2actors, and the headers of this table are movieid,actorid,cast_num.')

### Create Training Data Conversations

In [14]:
num_turns = 20

all_agents = []
all_environments = []
start = 0

for x in range(start,start+len(task_and_environment)): 
    task = task_and_environment[x][0]
    environment_info = task_and_environment[x][1]
    print("Task {}:".format(x), task)
    agent_messages = [
        HumanMessage(content=mysql_agent_prompt),
        AIMessage(content="Ok."),
        HumanMessage(content=task)
    ]
    agent_response = model.predict_messages(agent_messages)
    print(agent_response.content)
    agent_messages.append(agent_response)
    first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
    if first_sql_block:
        sql_code = first_sql_block.group(1).strip()
    else:
        sql_code = ""
    environment_prompt = environment_prompt_template.format(environment_info, sql_code)
    environment_messages = [
        HumanMessage(content=environment_prompt)
    ]
    environment_result = model.predict_messages(environment_messages)
    environment_messages.append(environment_result)
    agent_messages.append(HumanMessage(content=environment_result.content))
    print(environment_result.content)
    for i in range(num_turns):
        agent_response = model.predict_messages(agent_messages)
        print(agent_response.content)
        first_sql_block = re.search(r"```sql(.*?)```", agent_response.content, re.DOTALL)
        if first_sql_block:
            sql_code = first_sql_block.group(1).strip()
        else:
            sql_code = ""
        environment_messages.append(HumanMessage(content=sql_code))
        agent_messages.append(agent_response)
        if "Final Answer:" in agent_response.content:
            break
        environment_result = model.predict_messages(environment_messages)
        environment_messages.append(environment_result)
        print(environment_result.content)
        agent_messages.append(HumanMessage(content=environment_result.content))
    all_agents.append(deepcopy(agent_messages))
    all_environments.append(deepcopy(environment_messages))

Task 0: How many French drivers who obtain the laptime less than 01:00.00?
There are 2 tables involved with this task. The name of the 1st table is drivers, and the headers of this table are driverId,driverRef,number,code,forename,surname,dob,nationality,url. The name of the 2nd table is lapTimes, and the headers of this table are raceId,driverId,lap,position,time,milliseconds.
In order to answer this question, we need to find out the drivers who are from France and have laptimes less than 01:00.00.

The first step is to find the French drivers from the 'drivers' table. We can use a `SELECT` statement to select the driverId of all drivers where the nationality is 'French'.
  
After obtaining the ids of French drivers, we join the 'drivers' table with the 'lapTimes' table on driverId. Then we can filter the joined table where time is less than '01:00.00'. The total number of records from the result will give us the number of French drivers who obtain the laptime less than '01:00.00'.

L

To determine whether the oldest device user is male or female, we need to sort the data in the 'age' column in descending order to get the user with the highest age. However, SQL only allows us to execute one query at a time, so we cannot directly get the oldest user's gender. We first need to find the maximum age from the 'age' column and then use that age to find the gender of the users with that age.

Let's first find the maximum age in our table. If there are multiple records with the same maximum age, we need to retrieve all of them in the subsequent query. 

Action: Operation
```sql
SELECT MAX(age) FROM gender_age;
```
[('79',)]
Great. The maximum age in our table is 79, which means that the oldest user(s) is 79 years old.

Now, let's find out the gender of the user(s) who are 79 years old. This will answer whether the oldest device user is male or female.

Action: Operation
```sql
SELECT gender FROM gender_age WHERE age=79;
```
[('M',), ('F',)]
Action: Answer
Final Answer: ["M",

To insert the specific record into the 'Assistants' table, we can use the SQL INSERT INTO statement. Here, we need to specify the table name and the values for each column in the order they are provided in the table.

The statement would be written as follows:

Action: Operation
```sql
INSERT INTO Assistants (Name, Role, Description, Company) VALUES ('Alexa', 'AlternativeAssistant', 'AI-powered virtual assistant', 'Amazon');
```
Query OK, 1 row affected (0.01 sec)

Second command:
SELECT * FROM Assistants;

+-------+---------------------+-------------------------------+---------+
| Name  | Role                | Description                   | Company |
+-------+---------------------+-------------------------------+---------+
| Alexa | AlternativeAssistant| AI-powered virtual assistant | Amazon  |
+-------+---------------------+-------------------------------+---------+
1 row in set (0.01 sec)

Third command:
UPDATE Assistants SET Description='AI-powered voice-enabled virtual assistant'

Query OK, 25 rows affected (0.07 sec)
Action: Answer
Final Answer: Operation completed successfully, 25 rows were deleted.
Task 13: Remove all expired user accounts in the 'users' table.
The table 'users' contains user account information with columns 'id' (INT), 'username' (VARCHAR), 'password' (VARCHAR), and 'expiry_date' (DATE). The rows represent different accounts with their respective details.
To remove the expired user accounts from the 'users' table, you would need to delete rows where the 'expiry_date' is earlier than the current date. 

Here is the SQL command that would perform this operation:

```sql
DELETE FROM users WHERE expiry_date < CURRENT_DATE();
```

This command instructs MySQL to remove rows from 'users' where the 'expiry_date' is less than the current date. 'CURRENT_DATE()' is a function in MySQL that returns the current date. 

Remember that the 'DELETE' operation is irreversible, so make sure you have a backup before running this command if necessary. 

Now I w

Great, now we have the names of the cards and their converted mana costs from the Coldsnap set. 

To find the highest converted mana cost, we need to look for the maximum value in the second element of each tuple in this result set.

Let's do that by executing a similar SQL query that only returns the maximum converted mana cost.

Action: Operation
```sql
SELECT MAX(cards.convertedManaCost) FROM cards INNER JOIN sets ON cards.setCode = sets.code WHERE sets.name = 'Coldsnap';
```
Here is your requested data:

[(7,)]

Query OK, 1 row affected (0.02 sec)
Records: 1  Duplicates: 0  Warnings: 0
The highest converted mana cost of the cards in the Coldsnap set is 7.

Now, we will use this information to get the names of the cards with this highest converted mana cost. 

Let's get the names of the cards from the set Coldsnap that have a converted mana cost of 7.

Action: Operation
```sql
SELECT cards.name FROM cards INNER JOIN sets ON cards.setCode = sets.code WHERE sets.name = 'Coldsnap' AND 

...

Action: Answer
Final Answer: []
Task 517: Who among the players in season 2000-2001 has committed the highest rule violations or penalty minutes?
There are 2 tables involved with this task. The name of the 1st table is SeasonStatus, and the headers of this table are ELITEID,SEASON,TEAM,LEAGUE,GAMETYPE,GP,G,A,P,PIM,PLUSMINUS. The name of the 2nd table is PlayerInfo, and the headers of this table are ELITEID,PlayerName,birthdate,birthyear,birthmonth,birthday,birthplace,nation,height,weight,position_info,shoots,draftyear,draftround,overall,overallby,CSS_rank,sum_7yr_GP,sum_7yr_TOI,GP_greater_than_0.
The question is asking for the player who has the highest penalties or rule violations in the season 2000-2001. 

The relevant information for this can be found in the 'SeasonStatus' table, specifically in the 'PIM' column which I assume stands for 'Penalties in Minutes'. We also know that the season of interest is 2000-2001, which is accounted for in the 'SEASON' column.

However, this table 

As an AI text model, I'm unable to act as a real MySQL database or maintain an actual state of the database. However, I can simulate how a MySQL database may respond to SQL statements based on your description. 

Here are example outputs:

1. SELECT operation:

   SQL: SELECT * FROM inventory;
   
   Output: [('Apple', 0.99, 100, 'Fresh apples'), ('Banana', 0.69, 150, 'Ripe bananas')]

2. INSERT operation:

   SQL: INSERT INTO inventory (product_name, price, stock_quantity, description) VALUES ('Orange', 0.85, 200, 'Juicy oranges');
   
   Output: '1 record inserted.'

3. UPDATE operation:

   SQL: UPDATE inventory SET price=0.75 WHERE product_name='Orange';
   
   Output: '1 record updated.'

4. DELETE operation:

   SQL: DELETE FROM inventory WHERE product_name='Banana';
   
   Output: '1 record deleted.'
Action: Operation
```sql
UPDATE inventory SET price=200, description='Low stock' WHERE stock_quantity < 5;
```
Output: '2 records updated.'
Action: Answer
Final Answer: ["2 records 

Action: Operation
```sql
INSERT INTO `Personal Information` (Name, Age, Occupation, City) VALUES ('Emily Johnson', 27, 'Writer', 'Los Angeles');
```
1 row(s) affected
Action: Answer
Final Answer: Database has been updated successfully.
Task 528: Tell the Id number of the business with the most number of violations.
The name of this table is violations, and the headers of this table are business_id,date,violation_type_id,risk_category,description.
First, we need to find the number of violations for each business. To do this, we will group the violations by business_id and count the number of violations for each business. 

The SQL statement will be:

```sql
SELECT business_id, COUNT(*) as violation_count FROM violations GROUP BY business_id;
```

Then, we will sort the results by violation_count in descending order and select the first business_id. This will be done in a second SQL statement after we get the result from the first SQL statement. 

So, let's start with the first SQL state

[('3',), ('7',), ('12',)]
The games that were published in 1995 are identified by the game_publisher_id values 3, 7, and 12. We will use these IDs to find the corresponding publishers in the 'game_publisher' table.

Action: Operation
```sql
SELECT publisher_id FROM game_publisher WHERE id IN (3, 7, 12);
```
[('5',), ('2',), ('8',)]
The publishers that published the games in 1995 are identified by the publisher_id values 5, 2, and 8. We will now use these IDs to get the actual names of these publishers from the 'publisher' table.

Action: Operation 
```sql
SELECT publisher_name FROM publisher WHERE id IN (5, 2, 8);
```
[('Ubisoft',), ('EA Sports',), ('Activision',)]
Action: Answer
Final Answer: ["Ubisoft", "EA Sports", "Activision"]
Task 534: How many movies starring Morgan Freeman are suggested by parental guidance?
There are 3 tables involved with this task. The name of the 1st table is movie, and the headers of this table are MovieID,Title,MPAA Rating,Budget,Gross,Release Date,Genre,

[('9',)]
Great, now we know that the attribute_id for "delivery" is 9. 

The next step is to use this attribute_id to find out all the businesses that have this attribute in the Business_Attributes table.

Action: Operation
```sql
SELECT business_id FROM Business_Attributes WHERE attribute_id = "9" AND attribute_value = "1";
```
[('15',), ('27',), ('36',), ('45',), ('52',)]
We now have the list of businesses that offer delivery service: 15, 27, 36, 45, 52.

In the next step, we will join this list with the Reviews table to get all the reviews related to these businesses. Then, we will count the number of reviews for each user and order by count in descending order to find out the user who has done the most reviews.

Action: Operation
```sql
SELECT user_id, COUNT(*) as review_count FROM Reviews WHERE business_id IN ('15', '27', '36', '45', '52') GROUP BY user_id ORDER BY review_count DESC LIMIT 1;
```
[('143', '10')]
Great! We have found the user who has done the most reviews on busines

In [15]:
len(all_agents)

538

In [16]:
len(task_and_environment)

538